### GC Steps
Objects allocated using the constructor are stored in heap memory.  
![Heap](https://i.imgur.com/gSCevSI.png)

Over time as some objects become unreferenced. These objects are candidates for GC and can be removed to reclaim memory. The first step in GC is *marking* objects which can be removed.  
![Marking](https://i.imgur.com/KXyEIMB.png)  
Some ways that an object can become GC eligible are:
1. Nullifying reference variable:

```java
Person p1 = new Person("John Doe");
p1 = null; // the above created object is now eligible for GC
```


2. Reassigning:

```java
Person p1 = new Person("John Doe");
Person p2 = new Person("Jane Doe");
p1 = p2; // John Doe available for GC
```


3. Object created inside a method:

```java
public void someMethod(){
    Person p1 = new Person("John Doe");
}
// p1 now eligible for GC
```


4. Anonymous object:

```java
new Person("Steve Jobs").walk();
// Steve Jobs available for GC
```


5. Objects with just internal references:

<img src="https://miro.medium.com/max/875/0*XY9nzQPR3LXWWmZj" width="500" height="auto">



The next step is *deleting*. After deletion, we get lots of empty space.  
![Deletion](https://i.imgur.com/AJxJFmG.png)

The final step is *compacting* : referenced objects are moved together.  
![Compacting](https://i.imgur.com/TqoE9yo.png)

### Generational GC
In a typical program, most of the objects are short lived. Therefore it makes sense to divide the heap area into different parts depending upon age of objects.  
![GC Generations](https://i.imgur.com/r3fDW8P.png)

**Young Generation:** Newly created objects are put in Eden space in Young Generation. When Eden space fills up, it causes *minor garbage collection* to trigger. Objects that survive GC in Eden space are moved to Survivor space. The process called `mark-copy`:  
![Young generation GC process](https://i.imgur.com/7fqKOis.png)

**Old Generation:** After a minor GC, when aged objects reach a certain age threshold they are promoted from young generation to old generation. Garbage collection involving old generation heap is called *major garbage collection*. *Full GC* involves both old and young generation heap.

Both minor or major GC are stop the world events, meaning all program threads are paused and garbage collection thread runs.

### GC Implementations
When considering a certain GC implementation we look at the below two factors:
- Throughput: percentage of total time not spent in garbage collection considered over long periods of time. So if the application spends 10% of its time in garbage collection the throughput is 90%. A good GC implementation would maximise throughput.  
- Latency: GC operation pauses code execution. Latency measures pauses induced by the JVM as it performs garbage collection.

**Serial Garbage Collector:** This is the simplest GC implementation. This collection of garbage collectors uses `mark-copy` for the Young Generation and `mark-sweep-compact` for the Old Generation. As the name implies – both of these collectors are single-threaded collectors, incapable of parallelizing the task at hand. Both collectors also trigger stop-the-world pauses, stopping all application threads. To use this:
```
java -XX:+UseSerialGC com.mypackages.MyExecutableClass
```
Recommended only for the JVM with a couple of hundreds megabytes heap size, running in an environment with a single CPU.

**Parallel Garbage Collector:** This combination of Garbage Collectors uses mark-copy in the Young Generation and mark-sweep-compact in the Old Generation. Both Young and Old collections trigger stop-the-world events, stopping all application threads to perform garbage collection. Both collectors run marking and copying / compacting phases using multiple threads, hence the name ‘Parallel’. Using this approach, collection times can be considerably reduced. To use this:
```
java -XX:+UseParallelGC -XX:+UseParallelOldGC com.mypackages.MyExecutableClass
```
This collector result in high throughput and is suitable for multi-core machines. However, as all phases of the collection have to happen without any interruptions, these collectors are still susceptible to long pauses during which your application threads are stopped. So, not suitable for applications where latency is a concern.

**Concurrent Mark and Sweep Garbage Collector:** It uses the parallel stop-the-world mark-copy algorithm in the Young Generation and the mostly concurrent mark-sweep algorithm in the Old Generation. It does not compact the Old Generation but uses free-lists to manage reclaimed space. Secondly, it does most of the job in the mark-and-sweep phases concurrently with the application. It still competes for CPU time with the application threads. To use this:
```
java -XX:+UseConcMarkSweepGC com.mypackages.MyExecutableClass
```
Recommended on multi-core machines if your primary target is latency. Worse throughput than Parallel GC (mostly because not all threads are utilised).

**G1 Garbage Collector** Garbage-First is a soft real-time garbage collector, meaning that you can set specific performance goals to it. You can request the stop-the-world pauses to be no longer than x milliseconds within any given y-millisecond long time range, e.g. no more than 5 milliseconds in any given second. Garbage-First GC will do its best to meet this goal with high probability (but not with certainty, that would be hard real-time).